In [167]:
from nltk.tokenize import sent_tokenize, word_tokenize
import random
import nltk
from operator import itemgetter
import re
import numpy as np

In [168]:
def textProc(file_name):
    f= open(file_name,'r')
    #tp1
    corpus = f.read().replace('\n',' ') #replace the new line with single white space.
    corpus = corpus.lower() #tp2

    token = sent_tokenize(corpus)

    #replace all the non alphanumeric characters with whitespace.
    token = [re.sub(r'[^A-Za-z0-9]'," ", i) for i in token]#tp3

    random.shuffle(token)

    #addtG(token) #add tags
    return token

    #clean tokens of sentence returned


In [169]:
#write a function for count of n-gram
def LstOfNgram(count,corpus):
    #cleaned corpus
    n_gram=[]
    for i in corpus:
        p = word_tokenize(i)
        p = ["<s>"]+p
        p.append("</s>")    
        #token must have start and end line tags here.
        #print (p)
        for g in range(count-1,len(p)):
            dummy=""
            for j in range(count):
                dummy = p[g-j]+" "+dummy   #i,i-1,i-2,i-3,i-4,i-5,...,i-(n-1)
               #more things to be added here
            dummy = re.sub(r"^\s|\s$","",dummy)
            n_gram.append(dummy)
    return (n_gram)    

    

In [170]:
p = ["this is a sentence"]
o = LstOfNgram(3,p)
print (o)

['<s> this is', 'this is a', 'is a sentence', 'a sentence </s>']


In [171]:
def CountNgram(frequency):
    count ={}
    
    for (key,fvalue) in reversed(sorted(frequency.items(), key = itemgetter(1))):
        count[key] = fvalue
 
    return count

In [172]:
#unified method for MLE of n-gram
def MLE(a,b,N):  #n is N of n-gram
    try:
        if N ==1:#unigram
            return (count_unigram[a]/len(unigramLst))
        elif N==2:#bigram
            return (count_bigram[b+" "+a]/count_unigram[b])
        elif N == 3:#triram
            return (count_trigram[b+" "+a]/count_bigram[b])
        elif N == 4:
            return (count_quadgram[b+" "+a]/count_trigram[b])
    except:
            return 0 #0 probability if not present

In [173]:
def get_NLst(count_dict):#pass dicationary of word vs count.
    #create counting list for 
    l = count_dict 
    l=list(l.values())
    l.sort()
    #we want a dictionary with {1:1,2:1,3:2,4:3,5:1}
    d={}
    for i in range(len(l)):
        count =1
        for j in range(i,0,-1):
            if l[i] == l[j-1]:
                count = count + 1
            else:
                break 
    
        d[l[i]] = count
    return (d)
#takes O(n^2) which takes lot of time to execute.
    

In [174]:
class add1_smoothing_bigram:#old_count is the dictionary of count versus bigram/unigram
    
    def __init__(self,old_countUnigram, old_countBigram):
        self.old_countUnigram = old_countUnigram.copy()
        self.old_countBigram = old_countBigram.copy()
        self.new_countBigram = old_countBigram.copy()
#        print (self.new_countBigram)
        self.countBigram()
        
        
    def countBigram(self):
           
            for i in self.new_countBigram.keys():
                a_,b_= i.split(" ")[1],i.split(" ")[0]
                self.new_countBigram[i]=self.old_countUnigram[b_]*self.prob(a_,b_)
                
        
    def prob(self,a,b): #P(a|b)
            bigram = b+" "+a
            unigram = b
            try:
                numerator = self.old_countBigram[bigram] +1
            except :
                numerator = 1
            try:   
                denominator = self.old_countUnigram[unigram] + len(self.old_countUnigram.keys()) #count_unigram is count of unigram(types in corpus)
            except :
                denominator = len(self.old_countUnigram.keys())
            return (numerator/denominator)
#assure once that it is len(count_bigam)      ,token

In [175]:
class goodTurning:
    def __init__(self,count): 
        self.countNgram = count
        self.dictN = get_NLst(self.countNgram)
        self.keys = list(self.dictN.keys())    #[N1,N2,N3,N4,N5,N6,...]
        self.values = list(self.dictN.values()) #count of N1,N2,N3,...
        self.NewCount()
    def NewCount(self):
        self.newCount=self.countNgram.copy()
        
        for i in self.newCount.keys():
            try:
                self.newCount[i] = (self.newCount[i]+1)*(self.dictN[self.newCount[i]+1]/self.dictN[self.newCount[i]])
            except:
                intr= np.interp(float(self.newCount[i]),[float(j) for j in self.keys],[float(k) for k in self.values])
                #intr= np.interp(l[i]+1,keys,values)

                self.newCount[i] = (self.newCount[i]+1)*(intr/(self.dictN[self.newCount[i]]))
            #new for the word with zero count 

        return self.newCount 
    
    def smoothProb(self,a,token): 
        #token : total number of token of n-gram 
    
        if  a not in self.countNgram.keys():
            return (dictN[1]/len(token)) #count of 1 divided by total ngrams in the corpus
        
        else:
            return (self.NewCount()[a]/len(token))
#look over interpolation function
#for Nc >Nc+1 it is too jumpy
#the above probability may be wrong??
        

In [176]:

token = textProc('SherlockHomes.txt')


train = token[:int(len(token)*0.8)]#80%
test= token[int(len(token)*0.8):]#20%
#print (token)
#print (train)


In [177]:
#n-gram count
#token = ["this is a sentence","this is something differnt","this is something differnt that this has happend"]
#list contining n-gram
unigramLst = LstOfNgram(1,train)  ##the first number here representes the N of N-gram.
bigramLst  = LstOfNgram(2,train)
trigramLst = LstOfNgram(3,train)
quadgramLst= LstOfNgram(4,train)
#print (unigramLst)
#  query for count of unigram only once
feq  =  nltk.FreqDist(unigramLst)
freq =  nltk.FreqDist(bigramLst)
fre  =  nltk.FreqDist(trigramLst)
fr   =  nltk.FreqDist(quadgramLst)

#count_unigram={}
count_unigram = CountNgram(feq)
count_bigram = CountNgram(freq)
count_trigram = CountNgram(fre)
count_quadgram = CountNgram(fr)
print (count_quadgram["of this strange affair"])


1


# MLE for unigram, bigram, trigrams and quadgrams. 

In [178]:
l1 = MLE('affair','',1)
l2 = MLE('affair','strange',2)
l3 = MLE('affair','this strange',3)
l4 = MLE('affair','of this strange',4)
print (l1,l2,l3,l4)
#work to get the probability for a sentence

0.00013526449411079203 0.02702702702702703 0.3333333333333333 1.0


# Smoothing : ADD-1 

In [179]:
#QUESTION 5
ADD1 = add1_smoothing_bigram(count_unigram,count_bigram)
newProb = ADD1.prob("affair","strange")
#print (ADD1.new_countBigram)
print(newProb)
#work for sentence probability

0.0002798376941374003


# Smoothing : Good-turing

In [180]:
print (count_gram)

NameError: name 'count_gram' is not defined

In [ ]:
#n-gram evalutation, how many n-grams are there
#question 6 

p = goodTurning(count_bigram)
q = goodTurning(count_unigram)
#goodTuning smooting trained

In [ ]:
print (q.newCount)

##  Constant value discounting in good-tunning, for count b/w 1 and 10

In [ ]:
smooth_count = p.newCount
#print (smooth_count)
print (count_bigram["roylott had"] -smooth_count["roylott had"])#n=1
print (count_bigram['merry over'] - smooth_count['merry over'])#n=2
print (count_bigram['glass of'] - smooth_count['glass of'])
print (count_bigram['desire to'] - smooth_count['desire to'])
print (count_bigram['be taken'] - smooth_count['be taken'])
print (count_bigram['is this'] - smooth_count['is this'])
print (count_bigram['the bird'] - smooth_count['the bird'])#n =7
print (count_bigram['holmes said'] - smooth_count['holmes said'])
print (count_bigram['until i'] - smooth_count['until i'])
print (count_bigram['other side'] - smooth_count['other side'])
#the vaue of d is not constant in this corpus, after smoothing.

In [ ]:
smooth_count = q.newCount
#print (smooth_count)
print (count_unigram['suggests'] -smooth_count['suggests'])#n=1
print (count_unigram[ 'advanced'] - smooth_count[ 'advanced'])#n=2
print (count_unigram['discovering'] - smooth_count['discovering'])
print (count_unigram['remarks'] - smooth_count['remarks'])
print (count_unigram['carrying'] - smooth_count['carrying'])
print (count_unigram['belief'] - smooth_count['belief'])
print (count_unigram['solution'] - smooth_count['solution'])#n =7
print (count_unigram['finding'] - smooth_count['finding'])
print (count_unigram['path'] - smooth_count['path'])
print (count_unigram['feeling'] - smooth_count['feeling'])
#the vaue of d is not constant in this corpus, after smoothing.

In [ ]:
#question 7
#inverse of all the probability product of bigrams in the test data
def prePlexity(bigram_count,uigram_count): #for add1 they are the new counts
#the bigramData is a dictionary with count of all the bigrams.    
    N = len(bigram_count.keys())   #a dictionary
    p=1
    for i in bigram_count.keys:
        prob = bigram_count[i]/unigram_count[i.split(" ")[0]]
        p=p*prob
    perplex = (1/p)**(N)
    return perplex
        
    

In [ ]:
def perPlexity(data,n):#input data in form of sentence
    #give a data in form of list of sentence
    #N is the token
    prob = 1
    n=N
    for i in data:
        #i is a sentence
        p = LstOfNgram(2,[i])
        q = 1
        for j in p:  
           
            q = q*MLE(j.split(" ")[1],j.split(" ")[0],2)
           # print (q)
            if (q == 0):
                print (j)
        prob = prob*q
       # print (prob)
    perpLxty = (1/prob)**N
    return perPlexity
        
            
    

In [ ]:
l=[]
print (count_bigram["egg that"])
for i in test:
    l+=word_tokenize(i)
N = len(l)
perPlexity(test,N)


In [ ]:
#rough work below[a]/count_unigram

In [ ]:
l="<s>this is a sentence here is.</s>"
token = word_tokenize(l)
#print (token)
regx = re.compile("[a-z0-9]+ [a-z0-9]+")
r =re.findall(regx,l)
p= ["<s>this", "this is","is a ","a sentence","sentence here","here is","is. </s>"]
print (r)

In [ ]:



import re
l = "this is a sentence here is"
token1 = word_tokenize(l)
token1 = ["<s>"]+token1
token1.append("</s>")
n_gram = []
n=2 #bigram 
#so keeping a sentence token is a good idea
#for i in range(n-1,len(token)):#start with the n-1 of the n gram, so that it does not give any error
 #   n_gram.append(token[i-1]+" "+token[i])
#print (n_gram)

#for n gram              <s>this is a sentence here is</s>
print (token1)
for i in range(n-1,len(token1)):
    p=" "
    for j in range(n):
       p = token1[i-j]+" "+p   #i,i-1,i-2,i-3,i-4,i-5,...,i-(n-1)
       #more things to be added here
    n_gram.append(p)
#print (n_gram)

In [ ]:
print (test)

In [ ]:
1

In [ ]:
l={"this":4,"is":3,"my":4,"country":3,"hello":1,"bitch":2}



In [ ]:
l=[1,2,3,3,4,4,4,5,5,5,5,5,5,7,8,8,10,10,10,11,12]
l={"this":4,"is":3,"my":4,"country":3,"hello":1,"bitch":2}

l=list(l.values())
l.sort()

#we want a dictionary with {1:1,2:1,3:2,4:3,5:1}
d={}
for i in range(len(l)):
    count =1
    for j in range(i,0,-1):
        if l[i] == l[j-1]:
            count = count + 1
          
        else:
            break 
    
    d[l[i]] = count
d

In [ ]:
l = [""]

In [ ]:
print (p)

In [ ]:
l=[1,2,3,4,5]
p=l.copy()
p[0]= 2
p



In [ ]:
import numpy as np
l={"this":4,"is":3,"my":4,"country":3,"hello":1,"bitch":2}

dictN = get_NLst(l)#dictionary of all the N's and corresponding values

keys = list(dictN.keys())
values = list(dictN.values())
for i in l.keys():
    try:
       
        l[i] = (l[i]+1)*(dictN[l[i]+1]/dictN[l[i]])
    except:
        intr = np.interp(l[i]+1,keys,values)
        
        l[i] = (l[i]+1)*(intr/dictN[l[i]])
l


In [ ]:
l = ["<s>this is a sentence for me<\s>","<s>that is a sentence for you man<\s>"]
#inverse of all the bigram probability to the power N,
#probability of all the MLE birams
#define a function that takes the sentences from the test data set and give out ptobability.

